<center> <h1> Alternating Least Squares (ALS) with PySpark </h1> </center> <center> <h2> Gianvito Losapio  </h2> </center>

<br>
<center>  Large Scale Computing (101799) </center>
<center>  a.y. 2019/2020 </center>
<center> <b> Joint with: Matteo Ghirardelli, Sofia Bagnato, Federico Minutoli </b></center>
 


<br>

## Content

In this project, I will present Alternating Least Squares (ALS) algorithm in the contest of modern recommender systems. Here is the list of content:

- Recommender system overview with emphasis on collaborative filtering
- Description of the ALS algorithm
- Implementation of the ALS algorithm from scratch: sequential (ALS_local.py) and parallel version (ALS_parallel.py using Spark)
- Usage of ALS provided by Spark MLlib APIs to predict the ratings for the movies in [MovieLens Datasets](https://grouplens.org/datasets/movielens/latest/)

## Recommender systems
A recommendation system is basically an information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. It is widely used in different internet / online business such as Amazon, Netflix, Spotify, or social media like Facebook and Youtube. By using recommender systems, those companies are able to provide better or more suited products/services/contents that are personalized to a user based on his/her historical consumer behaviors

Recommender systems typically produce a list of recommendations and can be mainly categorized into:

- **Content-based filtering** analyzes the content (e.g., texts, metadata, features) of the item to identify related items. In particular, candidate items are compared with items previously rated by the user and the recommended items are similar to those that a user liked in the past.

- **Collaborative filtering** uses aggregated behavior/taste of a large number of users to suggest relevant items to specific users. Recommendations generated by CF are based solely on the user-user similarities (User-based Collaborative filtering, UBCF) and/or item-item similarities (Item-based Collaborative filtering, IBCF). For instance, UBCF is based on the assumption that people who agreed in the past will agree in the future, and that they will like similar kinds of items as they liked in the past. It follows that if, for example, a user $A$ has the same opinion as user $B$ on some item, then $A$ is more likely to have $B$’s opinion on some other item. 

- **Hybrid filtering**, which is a combination of collaborative filtering and content-based filtering, as the name suggests, could be more effective in some cases. Hybrid approaches can be implemented in several ways:
    - By making content-based and collaborative-based predictions separately and then combining them
    - By adding content-based capabilities to a collaborative-based approach (and vice versa)
    - By unifying the approaches into one mode.

## Collaborative filtering implementation

The starting point for collaborative filtering methods is the creation of a matrix $R$ to denote the past behavior of users, with size $n_u \times n_m$ where $n_u$ is the number of users and $n_m$ is the number of items (movies). $R_{ij}$ represents the rating score of movie $j$ rated by user $i$ with its value either being a real number (e.g. from 0.5 to 5) or missing (NaN). $R$ is known as **utility matrix**, **rating matrix** or **user-movie matrix**.


In many recommendation systems the aim of collaborative filtering is to estimate some of the missing values in $R$ (matrix completion problem) based on the known values through:
- the similarity between target users and other users (User-based Collaborative filtering, UBCF)
- the similarity between the items that target users interact with and other items (Item-based Collaborative filtering, IBCF)

The main problems of dealing with $R$ are size (large number of users, movies) and sparsity (it is impossible for all but a very few users to view lots of movies). Such problems can be alleviated through a low-rank approximation. This approach models both users and movies by giving them coordinates in a low dimensional feature space of fixed size $p$. Each user and each movie has a feature vector, and each rating (known or unknown) of a movie by a user is modeled as the inner product of the corresponding user and movie feature vectors. In other words, the factorization works by determining a feature vector for each user and product, such that the dot product of a user’s vector and a product’s is close to their score. More specifically

$$
R \approx U^{T}M
$$

$$
\left(\begin{array}{ccccc}
	& & & &\\
	& & & &\\
	& &R& & \\
	&  & & &\\
	& & & &\\
	\end{array}\right) \,\approx\,
\left(\begin{array}{ccc}
	& & \\
	& & \\
	& U^T& \\
	&  & \\
	& & \\
	\end{array}\right)
\,\,\left(\begin{array}{ccccc}
	& & & &\\
	& & M & &\\
	& & & & 
	\end{array}\right)
$$

where
- $U$ be the $p \times n_u$ user feature matrix, where each single column $u_i \in \mathbb{R}^{p}$ is the $p\times 1$ feature vector corresponding to user $i$ for all users $i = 1, \dots, n_u$
- $M$ be the $p \times n_m$ movie feature matrix, where each single column $m_j \in \mathbb{R}^{p}$ is the $p\times 1$ feature vector corresponding to movie $j$ for all movies $j = 1, \dots, n_m$.

Note that the dimension $p$ of the feature space is the number of hidden (latent) variables in the model. It is a system parameter that can be determined by a hold-out dataset or cross-validation.

If user ratings were fully predictable and $p$ sufficiently large, we could expect that $r_{ij} \approx < u_i, m_j >$, $\forall i, j$. In practice, the matrices $U$ and $M$ are obtained by solving an Empirical Risk Minimizaion (ERM) problem:

$$
\arg\min_{U,M} \dfrac{1}{n} \sum_{(i,j) \,\in\, I} \underbrace{(r_{ij} - < u_i, m_j >)^2}_{\ell(r, u, m)}
$$

where
- $I$ is the index set of the known ratings (not null) $I = \{(i,j) \,|\, r_{ij} \neq \emptyset\}$, of size $n$
- $\ell(r, u, m)$ is a loss function measuring the squared error between the true rating $r$ and the inner product $<u,m>$ used to approximate $r$.

In other words, the ideal objective is to minimize the mean square error between the known ratings $r_{ij}$ and the inner product $< u_i, m_j >$ used to approximate them. 

The solution consists of $(n_u + n_m) \times p$ parameters (the elements of $U$, $M$) to be determined. Solving the problem with many parameters (when $p$ is chosen relatively large) from a sparse dataset (when $n \lll n_u \times n_m$) usually overfits the data. To avoid overfitting, a common method appends a Tikhonov regularization term $T(U,M)$ to the empirical risk function

$$
\arg\min_{U,M} \dfrac{1}{n} \sum_{(i,j) \,\in\, I} (r_{ij} - < u_i, m_j >)^2 + \underbrace{\lambda\Big(\lVert U\Gamma_U \rVert^2 + \lVert M\Gamma_M \rVert^2\Big)}_{T(U,M)}
$$

where $\Gamma_U$ and $\Gamma_M$ are certain suitably selected Tikhonov matrices with the aim of penalizing large parameters.

The latent factors composing $U$ and $M$ are, hence, learned by minimizing the reconstruction error of the observed ratings. The unknown ratings can subsequently be computed by multiplying these predicted factors.

As a final remark, the problem has 2 hyperparameters to be tuned (e.g. with hold-out or cross-validation): the number of features (or latent variables) $p$ and the regularization parameter $\lambda$. 

## Alternating Least Squares

Alternating-Least-Squares with Weighted Regularization (ALS-WR) [[1]](https://endymecy.gitbooks.io/spark-ml-source-analysis/content/%E6%8E%A8%E8%8D%90/papers/Large-scale%20Parallel%20Collaborative%20Filtering%20the%20Netflix%20Prize.pdf) is one of the possible iterative algorithm (greedy solution) to solve the low rank approximation problem with the main advantage of being scalable to very large
datasets. ALS has been proposed within the Netflix Prize Competition, later widely studied and used in both academia and industry and implemented in many machine learning libraries (e.g. MLlib). It works as follows:

1. Initialize matrix $M$ by assigning the average rating for the movies as the first row (i.e., the first entry of each single feature vector), and small random numbers for the remaining entries;
2. Fix $M$, solve $U$ by minimizing the objective function;
3. Fix $U$, solve $M$ by minimizing the objective function similarly;
4. Repeat Steps 2 and 3 until a stopping criterion is satisfied.

The stopping criterion is based on the observed RMSEs on the validation set. After each round of updating both $U$ and $M$ (steps 2-3), if the improvement of the observed RMSE on the validation set is less than 1 bps (1e-4), the iteration stops and we use the obtained $U$,$M$ to make final predictions on the test set.

The following weighted regularization is proposed in the paper:

$$
\arg\min_{U,M} \sum_{(i,j) \,\in\, I} (r_{ij} - u_i^Tm_j)^2 + \lambda\Bigg(\sum_{i} n_{u_i} \lVert u_i \rVert^2 + \sum_{j} n_{m_j} \lVert m_j \rVert^2 \Bigg)
$$

where:
- $I$ is the index set of the known ratings (not null) $I = \{(i,j) \,|\, r_{ij} \neq$ NaN$\}$, of size $n$
- $u_i^Tm_j = < u_i, m_j >$ is the canonical inner product between the feature vectore of user $i$ ($u_i$) and the feature vectore of movie $j$ ($m_j$).
- $n_{u_i}$ is the number of ratings given by user $i$, that is the cardinality of the set $I_i$ of all the movies rated by user $i$, namely $I_i = \{(i,j) \in I \,|\,$ for a given $i\}$
- $n_{m_j}$ is the number of ratings for the movie $j$, that is the cardinality of the set of all the users who rated movie $j$, namely $I_j = \{(i,j) \in I \,|\,$ for a given $j\}$

This corresponds to Tikhonov regularization with matrices $\Gamma_U = diag(n_{u_i})$ and $\Gamma_M = diag(n_{m_j})$, being $diag(v)$ a diagonal matrix with elements of $v$ along the diagonal.

Note that problem
$$
\arg\min_{U,M} \lVert R - U^{T}M \rVert^2 + \lambda\Big(\lVert U\Gamma_U \rVert^2 + \lVert M\Gamma_M \rVert^2\Big)
$$
is not correct since the square error has to be computed only for the indicies $(i,j) \,\in\, I$. The other indicies are used for the prediction once the factorization is computed. A matrix formulation of the problem is not possible.

**Step 2**: Solve $U$ given $M$: each single column of $U$, say $u_i$ for $i = 1, \dots, n_u$, is considered as the $p\times 1$ weight vector solving the following regularized linear least squares problem:

$$
\arg\min_{u_i \in \mathbb{R}^p} \sum_{j \,\in\, I_i} (r_{ij} - u_i^Tm_j)^2 + \lambda n_{u_i} \lVert u_i \rVert^2 \qquad \forall \,i = 1, \dots, n_u
$$

where $j \in I_i$ are the (indicies of the) movies rated by user $i$, hence $r_{ij}$ are the known ratings of user $i$ and $m_j$ is the feature vectors of the movies that user $i$ has rated.

RLS is a well known strictly convex optimization problem, hence its solution is computed by setting the partial derivative w.r.t. $u_i$ of the argument equals to 0:

\begin{split}
& \sum_{j \,\in\, I_i} 2(r_{ij} - u_i^Tm_j)(-m_j) + 2\lambda n_{u_i}u_i = 0 \\
& \sum_{j \,\in\, I_i} -(r_{ij}\,m_j) + (u_i^T\,m_j)m_j + \lambda n_{u_i}u_i = 0 \\
& \sum_{j \,\in\, I_i} m_j\,m_j^T\,u_i + \lambda n_{u_i}u_i = \sum_{j \,\in\, I_i} r_{ij}\,m_j
\end{split}

Now, let:
- $M_{I_i}$ be the $p\times n_{u_i}$ submatrix of $M$ where columns $j \in I_i$ are selected (only feature vectors of the movies that user $i$ has rated)
- $R_{(i,I_i)}$ be the $1 \times n_{u_i}$ row vector obtained by selecting row $i$ and columns $j \in I_i$ from $R$ (all the ratings given by user $i$)
- $E_p$ be the $p\times p$ identity matrix

$$
\underbrace{\Big(M_{I_i}M_{I_i}^T + \lambda n_{u_i} E_p\Big)}_{A_i}u_i = \underbrace{M_{I_i}R_{(i,I_i)}^T}_{V_i}
$$

Hence, the final solution is:

$$
u_i = A_i^{-1}V_i
$$

for each column $i$ of $U$, $i = 1, \dots, n_u$.

**Step 3**: Solve $M$ given $U$: each single column of $M$, say $m_j$ for $j = 1, \dots, n_m$, is considered as the $p\times 1$ weight vector solving the following regularized linear least squares problem:

$$
\arg\min_{m_j \in \mathbb{R}^p} \sum_{i \in I_j} (r_{ij} - m_j^{T}u_i)^2 + \lambda n_{m_j} \lVert m_j \rVert^2 \qquad \forall \,j = 1, \dots, n_m
$$

Following the same procedure of step 2, letting

- $U_{I_j}$ be the $p\times n_{m_j}$ submatrix of $U$ where columns $i \in I_j$ are selected (only feature vectors of users who rated movie $j$)
- $R_{(I_j,j)}$ be the $n_{m_j} \times 1$ column vector obtained by selecting column $j$ and rows $i \in I_j$ from $R$ (all the ratings for the movie $j$)
- $E_p$ be the $p\times p$ identity matrix

we get

$$
\underbrace{\Big(U_{I_j}U_{I_j}^T + \lambda n_{m_j} E_p\Big)}_{A_j}m_j = \underbrace{U_{I_j}R_{(I_j,j)}}_{V_j}
$$

with the final solution being

$$
m_j = A_j^{-1}V_j
$$

for each column $j$ of $M$, $j = 1, \dots, n_m$.

**Summary:** ALS-WR approximates the solution to the low-rank approximation defined by the above Tikhonov regularization problem by solving least squares problem for each column of $U$, $M$ in an alternate fashion (hence the name): first $M$ is fixed and each single column of $U$ is computed as a solution to an independent RLS problem, then $U$ is fixed and each column of $M$ is computed analogously.

**On the regularization term:** Note that in solving each least squares problem the regularization parameter $\lambda$ is scaled by the number of ratings the user generated $n_{u_i}$ in updating user factors $u_i$, or the number of ratings the movie received $n_{m_j}$ in updating movie factors $m_j$ (this is why the method is called "ALS with weighted regularization"). It makes $\lambda$ less dependent on the scale of the dataset, so we can apply the best parameter learned from a sampled subset to the full dataset and expect similar performance. Using authors' words in the paper, this regularization never overfits the test data (empirically) when the number of features or number of iterations are increased. Moreover, the introduction of a regularization term is also necessary to "cure" matrices $A_i, A_j$ since they are very likely not invertible. 

### Complexity & parallel computation

Time complexity $c$:

$$
c \leq \delta \Bigg(\sum_{i=1}^{n_u} RLS_{u_i} + \sum_{j=1}^{n_m} RLS_{m_j}\Bigg)
$$

where $ RLS_{u_i} \approx O(p^2 n_{u_i} + p^3) $, $ RLS_{m_j} \approx O(p^2 n_{m_j} + p^3) $, $\delta$ is the maximum number of iterations.

Main problem: $n_u, n_m$ very large and fixed! Also heavy RLS cost if $p$ is chosen large. Swapping the elements in RLS complexity can be convenient if $p \gg n_{u_i}, n_{m_j}$.


Paper proposal: set of nodes with Matlab instances in a shared-nothing architecture.

1. Two distributed copies of $R$ are created in the cluster: the first copy is distributed by rows (i.e., by users), the second copy is distributed by columns (i.e., by movies). The distribution is by blocks of equal numbers of users or movies.

<div style="text-align:center"> <img src="./images/R_distr.png" alt="R distributed by rows" style="width:45%;"/> <figcaption><b></b></figcaption> </div>

2. Solve $U$ given $M$:
    - $M$ is initialized and replicated on all the nodes (only for the first round)
    - Each node computes (in parallel) $u_i$ for all users $i$ stored in its local memory ($R$ distributed by rows is used)
    - All the columns are gathered together (inter-node communication) and the resulting matrix $U$ is replicated on all the nodes.
    
<div style="text-align:center"> <img src="./images/u_i.png" style="width:40%;"/> <figcaption><b></b></figcaption> </div>
    
3. Solve $M$ given $U$:
    - Each node computes (in parallel) $m_j$ for all movies $j$ stored in its local memory ($R$ distributed by movies is used)
    - All the columns are gathered together (inter-node communication) and the resulting matrix $M$ is replicated on all the nodes.
    
<div style="text-align:center"> <img src="./images/m_j.png" style="width:40%;"/> <figcaption><b></b></figcaption> </div>  
    
4. Repeat Steps 2 and 3 until a stopping criterion is satisfied.

Result: almost linear speedup in the number of nodes with just estimated $\sim$5% of the run-time used for communication.

## ALS custom implementation with toy data

The custom parallel implementation of ALS (file ALS_parallel.py) has been developed according to the following algorithmic scheme:

<div style="text-align:center"> <img src="./images/als_algo_code.png" style="width:150%;"/> <figcaption><b></b></figcaption> </div>

A test of the algorithm on toy generated data follows.

In [10]:
from ALS_parallel import *

from pyspark.sql import SparkSession

# https://spark.apache.org/docs/latest/configuration.html#application-properties
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("ALS Naive") \
    .getOrCreate()

sc = spark.sparkContext

In [5]:
# Toy Rating Matrix

NaN = np.nan
R = np.array([[NaN, NaN, 4, 3, NaN],
              [3, NaN, NaN, NaN, 4],
              [NaN, 4, 3, 2, NaN], 
              [NaN, NaN, NaN, NaN, 3],
              [NaN, 2, 4, 5, 5],
              [4, NaN, 3, 3, NaN]])

print(R)

[[nan nan  4.  3. nan]
 [ 3. nan nan nan  4.]
 [nan  4.  3.  2. nan]
 [nan nan nan nan  3.]
 [nan  2.  4.  5.  5.]
 [ 4. nan  3.  3. nan]]


In [6]:
n_u, n_m = R.shape
print(n_u, n_m)

6 5


In [7]:
# Toy Validation Matrix

R_val = np.array([[NaN, 4, NaN, NaN, NaN],
              [NaN, NaN, NaN, 2, NaN],
              [NaN, NaN, NaN, NaN, NaN], 
              [NaN, 4, 3, NaN, NaN],
              [4, NaN, NaN, NaN, NaN],
              [NaN, NaN, NaN, NaN, 2]])

print(R_val)

[[nan  4. nan nan nan]
 [nan nan nan  2. nan]
 [nan nan nan nan nan]
 [nan  4.  3. nan nan]
 [ 4. nan nan nan nan]
 [nan nan nan nan  2.]]


In [8]:
# Cross-Validation

reg_params = [0.001, 0.1]
ranks = [3, 5]

min_delta = 0.001
max_iter = 10

best_U, best_M, min_rmse = CV_ALS_naive(R, R_val, min_delta, max_iter, reg_params, ranks, sc)

3 latent factors and regularization = 0.001: validation RMSE is 2.149898766873157
5 latent factors and regularization = 0.001: validation RMSE is 1.3714237845676285
3 latent factors and regularization = 0.1: validation RMSE is 1.2786557370058078
5 latent factors and regularization = 0.1: validation RMSE is 1.1972682136518045

The best model has 5 latent factors and regularization = 0.1


In [9]:
print(min_rmse)

1.1972682136518045


## ALS from MLlib on the MovieLens Dataset

MLlib includes an implementation of Alternating Least Squares (ALS) which scales well on clusters. It is located in the
`mllib.recommendation.ALS` class. The parameters accepted are:

- `numBlocks` is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
- `rank` is the number of latent factors in the model (defaults to 10).
- `maxIter` is the maximum number of iterations to run (defaults to 10).
- `regParam` specifies the regularization parameter in ALS (defaults to 0.1).
- `implicitPrefs` specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
- `alpha` is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).
- `nonnegative` specifies whether or not to use nonnegative constraints for least squares (defaults to false).



<br>

MLlib features a blocked implementation of the ALS algorithm that leverages Spark’s efficient support for distributed, iterative computation. It uses native LAPACK to achieve high performance and scales to billions of ratings on commodity clusters, as Fig. 3 shows.

In particular, MLlib offers a blocked implementation of the ALS factorization algorithm that groups the two sets of factors (referred to as “users” and “products”) into blocks and reduces communication by only sending one copy of each user vector to each product block on each iteration, and only for the product blocks that need that user’s feature vector. This is achieved by pre-computing some information about the ratings matrix to determine the “out-links” of each user (which blocks of products it will contribute to) and “in-link” information for each product (which of the feature vectors it receives from each user block it will depend on). This allows us to send only an array of feature vectors between each user block and product block, and have the product block find the users’ ratings and update the products based on these messages.

A test of the ALS algorithm from MLlib on the MovieLens dataset follows (executed on local machine). The files ALS_utils.py and io.py in the folder utils contain user-defined functions for data loading, cross validation, prediction, performance computation and data saving.

In [11]:
from pyspark.sql import SparkSession

import time

# Custom import
from utils.ALS_utils import holdout_cv_ALS, predict, compute_rmse, compute_accuracy
from utils.io import load_data, save_predictions

# -----------------------------------------------------

# SPARK SESSION
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("ALS Local") \
    .getOrCreate()

In [13]:
# INPUT CSV PATH
uri_train = "../data/training.csv"
uri_test = "../data/test.csv"

# Load and split
training_df, validation_df, test_df = load_data(spark, uri_train, uri_test)

print('Training size: {}'.format(training_df.count()))
print('Validation size: {}'.format(validation_df.count()))
print('Test size: {}'.format(test_df.count()))

Training size: 9358942
Validation size: 1986102
Test size: 2419652


In [14]:
# CROSS-VALIDATION
num_iterations = 5
ranks = [3, 5]
reg_params = [0.001, 0.01]


print('-------------------------------------------')
print('Starting Hold-out Cross Validation ...')
start_time = time.time()
best_model, errors = holdout_cv_ALS(training_df, validation_df, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

-------------------------------------------
Starting Hold-out Cross Validation ...
3 latent factors and regularization = 0.001: validation RMSE is 0.914144619536509
5 latent factors and regularization = 0.001: validation RMSE is 0.9195482316800119
3 latent factors and regularization = 0.01: validation RMSE is 0.8848426531891885
5 latent factors and regularization = 0.01: validation RMSE is 0.8902666779191226

The best model has 3 latent factors and regularization = 0.01
Total Runtime: 2227.18 seconds


In [15]:
# Predictions
num_movies = 10
predictions_df = predict(best_model, num_movies)


# RMSE, Accuracy
rmse_val = compute_rmse(best_model, validation_df)
rmse_test = compute_rmse(best_model, test_df)
accuracy_val = compute_accuracy(predictions_df, validation_df)
accuracy_test = compute_accuracy(predictions_df, test_df)
print('Validation RMSE: {}'.format(rmse_val))
print('Test RMSE: {}'.format(rmse_test))
print('Validation Accuracy: {}'.format(accuracy_val))
print('Test Accuracy: {}'.format(accuracy_test))

Validation RMSE: 0.8848426531891885
Test RMSE: 0.9390683889663092
Validation Accuracy: 1.9803991265932063E-5
Test Accuracy: 4.413998635409827E-5


### Execution on the cluster

Results from the execution on the cluster are reported.

In [ ]:
# CROSS-VALIDATION
num_iterations = 10
ranks = [8, 14, 20, 50]
reg_params = [0.001, 0.01, 0.1]


#  OUTPUT

#  Training size: 12834853 
#  Validation size: 2728925 
#  Test size: 3309723 

# -------------------------------------------
#   Starting Hold-out Cross Validation ...
#
#   8 latent factors and regularization = 0.001: validation RMSE is 0.932873813752 
#   14 latent factors and regularization = 0.001: validation RMSE is 1.00197091393 
#   20 latent factors and regularization = 0.001: validation RMSE is 1.07104564274 
#   50 latent factors and regularization = 0.001: validation RMSE is 1.24337729733 
#   8 latent factors and regularization = 0.01: validation RMSE is 0.906121507214 
#   14 latent factors and regularization = 0.01: validation RMSE is 0.954053124696 
#   20 latent factors and regularization = 0.01: validation RMSE is 0.986462340501 
#   50 latent factors and regularization = 0.01: validation RMSE is 1.0444401692 
#   8 latent factors and regularization = 0.1: validation RMSE is 0.852232077504 
#   14 latent factors and regularization = 0.1: validation RMSE is 0.847765112032 
#   20 latent factors and regularization = 0.1: validation RMSE is 0.846224028702 
#   50 latent factors and regularization = 0.1: validation RMSE is 0.845800476809 
  
#   The best model has 50 latent factors and regularization = 0.1 
#   Total Runtime: 8581.58 seconds 
#   Validation RMSE: 0.845800476809 
#   Test RMSE: 0.891425416072 
#   Validation Accuracy: 2.8116699099525717E-4 
#   Test Accuracy: 2.658729865660288E-4 

Cluster details: 8 machines with the following resources: (i) Operating system: Ubuntu 18.04; (ii) Storage: 250 GB; (iii)
CPU: 8 core; (iv) RAM: 32 GB.

Execution time for the cross-validation: 8581.58 seconds $\approx$ 143 mins $\approx$ 2 hours and 20 mins

### On the performance metrics

Let $R^*$ be the $n_u \times n_m$ rating matrix used as test set (compatibility with the rating matrix size used for training, e.g. achieved through a cold start strategy).

**RMSE**: Root mean square error between the known ratings in the test set and the corresponding predictions 

$$
RMSE = \sqrt{\dfrac{1}{|I|} \sum_{(i,j) \,\in\, I} (r_{ij} - \hat{r_{ij}})^2}
$$

where:
- $I$ is the index set of the known ratings in $R^*$
- $\hat{r_{ij}}$ is the prediction, namely $u_i^Tm_j$ in case of ALS 

**Accuracy metric** : Let $T$ be the $n_u \times n_t$ matrix where each row contains the $n_t$ top recommendations for each user, according to the predictions obtained on $R^*$. The accuracy is then defined as:

$$
accuracy = \dfrac{1}{n_u} \sum_{i = 1}^{n_u} \Bigg( \dfrac{1}{\min (n_t, |L_i|)} \sum_{k = 1}^{\min (n_t, |L_i|)} \mathbb{1}{\{t_{ik} \in L_i\}\Bigg)}
$$

where:

- $t_{ik} \in T$ is the $k$-th top recommendation for user $i$
- $L_i$ is the set of movies that user $i$ liked, namely $L_i = \{j \,|\, R_{ij}^* \geq 3\}$

ALS is designed to minimize RMSE through a probabilistic matrix factorization, hence a good result in RMSE is expected. On the other hand, ALS makes no use of other information (e.g. user preferences, genres, popular movies) and a very low accuracy is also expected on a large dataset. As a conclusion, ALS is good for an overall forecasting of the ratings but has to be supported with other filtering criteria to provide meaningful recommendations.